In [2]:
import numpy as np
import pandas as pd
import random
import torch
import os
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("friedrichor/MSR-VTT", "train_7k")['train']

D:\New folder\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(ds)
print(ds.column_names)
print(ds.features)
sampled_ds = ds.shuffle(seed=seed).select(range(700))
print(sampled_ds)
data_path = os.path.join("data", "MSR-VTT")
os.makedirs(data_path, exist_ok=True)
sample = sampled_ds[0]
print(sample["video_id"])
print(sample["caption"][:10])   # first ten captions
print(sample["url"])
print(sample["start time"], sample["end time"])
out = os.path.join(data_path, f"{sample['video_id']}.mp4")
print(out)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 7010
})
['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id']
{'video_id': Value('string'), 'video': Value('string'), 'caption': List(Value('string')), 'source': Value('string'), 'category': Value('int64'), 'url': Value('string'), 'start time': Value('float64'), 'end time': Value('float64'), 'id': Value('int64')}
Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 700
})
video6739
['a clip of young woman singing on a stage', 'a group of women are singing a song', 'a group of women is performing a song live', 'a group of women singing on stage', 'a video of five women singing live', 'an all girl band sings on a live tv show', 'five female vocalists perform live', 'five ladies perform a song for a tv show', 'five women singing live on a s

In [7]:
import subprocess
failed_count = 0
def download_clip(clip_info):
    # Download and trim using ffmpeg
    url =  clip_info['url']
    start = 0
    end = clip_info['end time'] - clip_info['start time']
    out = os.path.join(data_path, f"{clip_info['video_id']}.mp4")
    out = out.replace("\\", "/")
    global failed_count
    failed_count += 1
    ckp = r"D:\thanhloc\Semester251\DIP&CV\yt_cookies.txt"
    if os.path.exists(out):
        print(f"File {out} already exists, skipping download.")
        return out
    try:
        cmd = [
            "yt-dlp", "--cookies", ckp, "--sleep-interval", "1", "--max-sleep-interval", "3",
            "-f", "bestvideo[height<=480]*+bestaudio/best[height<=480]",
            "-o", out,
            url
        ]
        subprocess.run(cmd, check=True)
        return out
    except Exception as e:
        print(f"Failed {clip_info['video_id']}: {e}")
        return ""

In [8]:
sampled_ds = sampled_ds.map(
    lambda x: {"clip_path": download_clip(x)},
)
print(failed_count)
print(sampled_ds)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

File data/MSR-VTT/video6739.mp4 already exists, skipping download.
File data/MSR-VTT/video975.mp4 already exists, skipping download.


Map:   0%|          | 3/700 [00:03<11:49,  1.02s/ examples]

Failed video2663: Command '['yt-dlp', '--cookies', 'D:\\thanhloc\\Semester251\\DIP&CV\\yt_cookies.txt', '--sleep-interval', '1', '--max-sleep-interval', '3', '-f', 'bestvideo[height<=480]*+bestaudio/best[height<=480]', '-o', 'data/MSR-VTT/video2663.mp4', 'https://www.youtube.com/watch?v=Nwt-DUpKPYA']' returned non-zero exit status 1.
File data/MSR-VTT/video143.mp4 already exists, skipping download.
File data/MSR-VTT/video5587.mp4 already exists, skipping download.


Map:   1%|          | 6/700 [00:05<10:22,  1.11 examples/s]

...

Failed video1225: Command '['yt-dlp', '--cookies', 'D:\\thanhloc\\Semester251\\DIP&CV\\yt_cookies.txt', '--sleep-interval', '1', '--max-sleep-interval', '3', '-f', 'bestvideo[height<=480]*+bestaudio/best[height<=480]', '-o', 'data/MSR-VTT/video1225.mp4', 'https://www.youtube.com/watch?v=X_S0Nx-fCHo']' returned non-zero exit status 1.


Map: 100%|██████████| 700/700 [21:41<00:00, 10.72s/ examples]

Failed video6171: Command '['yt-dlp', '--cookies', 'D:\\thanhloc\\Semester251\\DIP&CV\\yt_cookies.txt', '--sleep-interval', '1', '--max-sleep-interval', '3', '-f', 'bestvideo[height<=480]*+bestaudio/best[height<=480]', '-o', 'data/MSR-VTT/video6171.mp4', 'https://www.youtube.com/watch?v=TS77oMc7l2Q']' returned non-zero exit status 1.


Map: 100%|██████████| 700/700 [21:42<00:00,  1.86s/ examples]

700
Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id', 'clip_path'],
    num_rows: 700
})


In [10]:
sample = sampled_ds[300]
print(sample["video_id"])
print(sample["caption"][:10])   # first ten captions

print(sample["clip_path"])

video6544
['a drama on the stage show', 'a group of indian men giving a talk to a group', 'a man is holding a hat', 'a man is talking to a man about the audience', 'a man is talking', 'a military official and two young men are talking in front of a group of people in a non-english language', 'a spanish gameshow on tv', 'about a malayalam tv show', 'guys on stage for live tv show', 'man in military outfit talks with other men']
data/MSR-VTT/video6544.mp4


In [11]:
missing_ds = sampled_ds.filter(lambda x: not x.get("clip_path"))
print(missing_ds)
print(sampled_ds)

Filter: 100%|██████████| 700/700 [00:00<00:00, 4594.06 examples/s]

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id', 'clip_path'],
    num_rows: 202
})
Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id', 'clip_path'],
    num_rows: 700
})


In [12]:
sampled_ds = sampled_ds.filter(
    lambda x: x.get("clip_path") not in ("", None)
)
print("Rows after drop:", len(sampled_ds))
print(sampled_ds)

Filter: 100%|██████████| 700/700 [00:00<00:00, 11744.33 examples/s]

Rows after drop: 498
Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id', 'clip_path'],
    num_rows: 498
})


In [13]:
sampled_ds.to_csv("data/sampled_msrvtt_clips.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.85ba/s]


571581